In [5]:
from bs4 import BeautifulSoup
import json
from tqdm import tqdm
import pandas as pd
import re

import sys
sys.path.append('../../')
from onemap_client import OneMapClient
import os
from dotenv import load_dotenv
load_dotenv()
tqdm.pandas()

In [23]:
files = [x for x in os.listdir('./data/raw/20231116/listing_page') if x!='.DS_Store']
store = {}
dups = set()

for file in tqdm(files):
    try:
        with open(f'./data/raw/20231116/listing_page/{file}', 'r') as f:
            page = BeautifulSoup(f, 'html.parser')
    except:
        print(file)
        break
    
    property_guru_id = file.strip('.html')

    info = {
        'listed_on': None, 
    }
    id_on_page = None
    try:
        if page.find(name='div', attrs={'class':'error-page-view-root'}) or page.find(name='div', attrs={'class':'property-overview-section'})==None:
            dups.add(file)
            continue
        more_details = page.find(name='section', attrs={'data-automation-id': 'more-details-widget'})
        detail_rows = more_details.find_all(name='td')
        for row in detail_rows:
            label = row.find(name='div', attrs={'class':'meta-table__item__label'}).text.strip()
            value = row.find(name='div', attrs={'class':'meta-table__item__value'}).text.strip()
            if label=='Listed On':
                info['listed_on'] = value
            if label=='Listing ID':
                id_on_page = value
        
        if id_on_page!=None and id_on_page!=property_guru_id:
            print(property_guru_id)
            break
        store[property_guru_id] = info
    except:
        print(file)
        break

100%|██████████| 612/612 [00:29<00:00, 20.41it/s]


In [29]:
with open('./data/processed/20231116/raw_extracted_listing_page.json', 'w') as f:
    json.dump(store, f)